In [1]:
!pip install transformers --upgrade
# !pip install datasets
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install sklearn
#!pip install scikit-learn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cached-path 1.1.6 requires huggingface-hub<0.11.0,>=0.8.1, but you have huggingface-hub 0.12.1 which is incompatible.
allennlp 2.10.1 requires transformers<4.21,>=4.1, but you have transformers 4.26.1 which is incompatible.


In [2]:
from transformers import BertTokenizer,BertModel,BertConfig
from transformers import ErnieModel
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import RobertaTokenizer, RobertaModel
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import logging
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/kaggle/input/edos-dev-task/EDA_train_5.csv',engine='python')
                 #,names=['rewire_id','text','label_sexist','label_category','label_vector','label_sexist0','sentiment'])
#df = pd.read_csv('/kaggle/input/edos-dev-task/train_all_tasks.csv',engine='python')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rewire_id  80000 non-null  object
 1   text       80000 non-null  object
 2   label      80000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.8+ MB


In [4]:
df['label'].value_counts()

#df['label_sexist'].value_counts()

0    60580
1    19420
Name: label, dtype: int64

In [5]:
# df.loc[df['label_sexist']=='not sexist','label_sexist']=0
# df.loc[df['label_sexist']=='sexist','label_sexist']=1
file1=np.array(df)

# df.loc[df['label_sexist']=='not sexist','label_sexist']=0
# df.loc[df['label_sexist']=='sexist','label_sexist']=1

# df.loc[df['label']=='not sexist','label_sexist']=0
# df.loc[df['label']=='sexist','label_sexist']=1


file1=np.array(df)
x=[x[1] for x in file1]
y=[int(x[2]) for x in file1]
z=[x[0][19:] for x in file1]
#print(x,y,z)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, z_train, z_test =  train_test_split(x, y, z, test_size=0.1)
#z_test

In [7]:
#df_dev=pd.read_csv('/kaggle/input/edos-dev-task/EDA_test_task_a_5.csv',encoding="unicode_escape",usecols=['rewire_id','text'])
df_dev=pd.read_csv('/kaggle/input/edos-dev-task/dev_task_a_entries.csv')
#df_dev=pd.read_csv('/kaggle/input/edos-dev-task/test_task_a_entries.csv')
df_dev.info()
#df['label_pred'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rewire_id  2000 non-null   object
 1   text       2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [8]:
x_dev= df_dev['text'].tolist()
z_dev=[int(x[19:]) for x in list(df_dev['rewire_id'])]
#print(len(z_train))

In [9]:
#tokenizer = BertTokenizer.from_pretrained("nghuyong/ernie-2.0-large-en")
#tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encoding = tokenizer(x_train, truncation=True, padding=True, max_length=50)
test_encoding = tokenizer(x_test, truncation=True, padding=True, max_length=50)
dev_encoding=tokenizer(x_dev,truncation=True,padding=True, max_length=50)

In [10]:
print(train_encoding.keys())

dict_keys(['input_ids', 'attention_mask'])


## 数据集封装和处理

In [11]:
# 数据集读取, 继承torch的Dataset类，方便后面用DataLoader封装数据集
class NewsDataset(Dataset):
    def __init__(self, encodings, labels,ids):
        self.encodings = encodings
        self.labels = labels
        self.ids=ids
    
    #这里的idx是为了让后面的DataLoader成批处理成迭代器，按idx映射到对应数据
    def __getitem__(self, idx):
        item = {key: torch.as_tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.as_tensor(int(self.labels[idx]))
        item['ids']=torch.as_tensor(int(self.ids[idx]))
        return item
    
    #数据集长度。通过len(这个实例对象)，可以查看长度
    def __len__(self):
        return len(self.labels)
    
class DevDataset(Dataset):
    def __init__(self, encodings,ids):
        self.encodings = encodings
        self.ids=ids
    
    #这里的idx是为了让后面的DataLoader成批处理成迭代器，按idx映射到对应数据
    def __getitem__(self, idx):
        item = {key: torch.as_tensor(val[idx]) for key,val in self.encodings.items()}
         #item['encodings']=torch.as_tensor(self.encodings[idx])
        item['ids']=torch.as_tensor(self.ids[idx])
        return item
    
    #数据集长度。通过len(这个实例对象)，可以查看长度
    def __len__(self):
        return len(self.ids)
    
#将数据集包装成torch的Dataset形式
train_dataset = NewsDataset(train_encoding, y_train,z_train)
test_dataset = NewsDataset(test_encoding, y_test,z_test)
dev_dataset=DevDataset(dev_encoding,z_dev)
# 单个读取到批量读取
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)
dev_dataloader=DataLoader(dev_dataset,batch_size=16,shuffle=True)

In [12]:

class Model(torch.nn.Module):
    def __init__(self, checkpoint="bert-large-uncased"):
        super().__init__()
        #self.pretrained = ErnieModel.from_pretrained("nghuyong/ernie-2.0-large-en")
        #self.pretrained = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")
        self.pretrained = RobertaModel.from_pretrained('roberta-base')
        self.fc = torch.nn.Sequential(torch.nn.Linear(768, 2))
 
    def forward(self, input_ids, attention_mask):
        logits = self.pretrained(input_ids=input_ids, attention_mask=attention_mask)
        logits = logits.last_hidden_state[:, 0]
        logits = self.fc(logits)
        return logits
    
# class Config(object):

#     def __init__(self):
#         self.pre_bert_path="nghuyong/ernie-1.0"
#         self.train_path = 'data/dataset_train.csv'  # 训练集
#         self.dev_path = 'data/dataset_valid.csv'  # 验证集
#         self.test_path = 'data/test.csv'  # 测试集
#         self.class_path = 'data/class.json'  # 类别名单
#         self.save_path ='mymodel/ernie.pth'        # 模型训练结果
#         self.num_classes=10
#         self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # 设备

#         self.epochs = 10  # epoch数
#         self.batch_size = 128  # mini-batch大小
#         self.maxlen = 32  # 每句话处理成的长度(短填长切)
#         self.learning_rate = 5e-4                                       # 学习率
#         self.hidden_size=768
#         self.tokenizer = AutoTokenizer.from_pretrained(self.pre_bert_path)

# class Model(nn.Module):
#     def __init__(self, config):
#         super(Model, self).__init__()
#         self.ernie=AutoModel.from_pretrained(config.pre_bert_path)
#         #设置不更新预训练模型的参数
#         for param in self.ernie.parameters():
#             param.requires_grad = False
#         self.fc = nn.Linear(config.hidden_size, config.num_classes)
#     def forward(self, input):
#         out=self.ernie(input_ids =input['input_ids'],attention_mask=input['attention_mask'],token_type_ids=input['token_type_ids'])
#         #只取最后一层CLS对应的输出
#         out = self.fc(out.pooler_output)
#         return out

## 实例化模型、定义损失函数和优化器

In [13]:
from torch.autograd import Variable
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device,'能用')


model = Model().to(device)

class FocalLoss(nn.Module):
    #def __init__(self, class_num, alpha=[0.33,0.67], gamma=2, use_alpha=True, size_average=True):
    #8360
    def __init__(self, class_num, alpha=[0.34,0.66], gamma=2, use_alpha=True, size_average=True):
    #83.6
    #def __init__(self, class_num, alpha=[0.35,0.65], gamma=2, use_alpha=True, size_average=True):
    #83.2
        super(FocalLoss, self).__init__()
        self.class_num = class_num
        self.alpha = alpha
        self.gamma = gamma
        if use_alpha:
            self.alpha = torch.tensor(alpha).cuda()
        self.softmax = nn.Softmax(dim=1)
        self.use_alpha = use_alpha
        self.size_average = size_average
    def forward(self, pred, target):
        prob = self.softmax(pred.view(-1,self.class_num))
        prob = prob.clamp(min=0.0001,max=1.0)
        target_ = torch.zeros(target.size(0),self.class_num).cuda()
        target_.scatter_(1, target.view(-1, 1).long(), 1.)
        if self.use_alpha:
            batch_loss = - self.alpha.double() * torch.pow(1-prob,self.gamma).double() * prob.log().double() * target_.double()
        else:
            batch_loss = - torch.pow(1-prob,self.gamma).double() * prob.log().double() * target_.double()
        batch_loss = batch_loss.sum(dim=1)
        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss

criterion = FocalLoss(class_num=2)
#criterion = nn.CrossEntropyLoss().to(device)

# 优化方法
#过滤掉被冻结的参数，反向传播需要更新的参数
optim = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
total_steps = len(train_loader) * 1
scheduler = get_linear_schedule_with_warmup(optim, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

cuda 能用


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  Futu

## 定义训练函数

In [14]:
from statistics import mean
def calculate_f1(stacked):
    stacked=torch.as_tensor(stacked,dtype=torch.int64)
    cmt=torch.zeros(2,2,dtype=torch.int32)
    #cmt
    for p in stacked:
        tl,pl=p.tolist()
        cmt[tl,pl]=cmt[tl,pl]+1
    M=cmt
    n = len(M)
    pre_all=[]
    rec_all=[]
    f1_all=[]
    sum_acc=0
    for i in range(len(M[0])):
        rowsum, colsum = sum(M[i]), sum(M[r][i] for r in range(n))
        try:
            precision=float(M[i][i]/float(colsum))
            pre_all.append(precision)
            print(float(M[i][i]))
            print(float(colsum))
            recall=float(M[i][i]/float(rowsum))
            rec_all.append(recall)
            f1=2*(precision*recall)/(precision+recall)
            f1_all.append(f1)

            sum_acc+=M[i][i].item()
        except ZeroDivisionError:
            print('precision: %s' % 0, 'recall: %s' % 0)
    print(cmt)
    accuracy=sum_acc/((sum(M).sum()).item())
    print('accuracy:%s' % accuracy ,'precision: %s' % (mean(pre_all)), 'recall: %s' % (mean(rec_all)),'F1: %s ' % (mean(f1_all)))

In [15]:
import numpy as np
import pandas as pd

def save_to_csv(stacked):
    stacked=torch.as_tensor(stacked).cpu()
    t_np = stacked.numpy() #convert to Numpy array
    df = pd.DataFrame(t_np) #convert to a dataframe
    
    df.columns=['rewire_id', 'label_pred']
    df['rewire_id']=['sexism2022_english-'+str(x) for x in df['rewire_id']]
    
    df.loc[df['label_pred']==0,'label_pred']='not sexist'
    df.loc[df['label_pred']==1,'label_pred']='sexist'
    

    print(df)
    df.to_csv("/kaggle/working/test_a_1.csv",index=False) #save to file
    print("保存成功！")
    #Then, to reload:

In [16]:
# 精度计算
#imports untils
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
batch_size=16
def validation():
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    with torch.no_grad():
        results=torch.zeros(0).to(device)
        label_all=torch.zeros(0).to(device)
        id_all=torch.zeros(0).to(device)
        for batch in test_dataloader:
            # 正常传播
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            ids=batch['ids'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            total_eval_loss += loss.item()
            logits = outputs
            
            # 获取预测结果
            preds = outputs.argmax(1).to(device)
            results=torch.cat((results,preds),dim=0)#预测的结果
            
            label_all=torch.cat((label_all,labels),dim=0)
            id_all=torch.cat((id_all,ids),dim=0)
            
        stacked=torch.stack(
                (
                    torch.as_tensor(label_all).to(device),
                    torch.as_tensor(list(map(int,results))).to(device)
                )
            )
        stacked=stacked.t()
        calculate_f1(stacked)
        
#         stacked_saved=torch.stack(
#                     (
#                         torch.as_tensor(list(map(int,id_all))).to(device),
#                         torch.as_tensor(list(map(int,label_all))).to(device),#真实结果
#                         torch.as_tensor(list(map(int,results))).to(device)#预测结果
#                     )
#                 )
#         stacked_saved=stacked_saved.t()
#         save_to_csv(stacked_saved)
        
        #print(stacked_saved)
    print("Average testing loss: %.4f"%(total_eval_loss/len(test_dataloader)))
    print("-------------------------------")

In [17]:
def train():
    batch_size=16
    for epoch in range(1):
        print("------------Epoch: %d ----------------" % epoch)
        train_loss = 0.0 # 训练损失
        val_loss = 0.0 # 验证损失
        model.train() # 声明开始训练
        total_train_loss = 0
        iter_num = 0
        total_iter = len(train_loader)
        
        results=torch.zeros(0).to(device)
        label_all=torch.zeros(0).to(device)
        labels=torch.zeros(0).to(device)
        for batch in train_loader:
            # 正向传播
            optim.zero_grad()
            #print(batch)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            ids=batch['ids'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)

            loss = criterion(outputs, labels)    

            total_train_loss += loss.item()

            loss.backward()# 反向传播
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #梯度裁剪，防止梯度爆炸  
            
            # 获取预测结果
            preds = outputs.argmax(1).to(device)
            
            
            stacked=torch.stack(
                    (
                        ids,
                        labels,
                        preds
                    )
                )
            stacked=stacked.t()
            #save_to_csv(stacked)
            

            # 参数更新
            optim.step()
            scheduler.step()

            iter_num += 1
            if(iter_num % 100==0):
                print("epoth: %d, iter_num: %d, loss: %.4f, %.2f%%" % (epoch, iter_num, loss.item(), iter_num/total_iter*100))
        print("Epoch: %d, Average training loss: %.4f"%(epoch, total_train_loss/len(train_loader)))     
                
        validation()
        
        model.eval()
        print("--------------开始分类dev集-----------------")
        with torch.no_grad():
            results=torch.zeros(0).to(device)
            #label_all=torch.zeros(0).to(device)
            ids_all=torch.zeros(0).to(device)
            for batch in dev_dataloader:
                # 正常传播
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                #labels = batch['labels'].to(device)
                ids=batch['ids'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                

                # 获取预测结果
                preds = outputs.argmax(1).to(device)
                results=torch.cat((results,preds),dim=0)#预测的结果
                ids_all=torch.cat((ids_all,ids),dim=0)

            stacked_saved=torch.stack(
                        (
                            torch.as_tensor(list(map(int,ids_all))).to(device),
                            torch.as_tensor(list(map(int,results))).to(device)#预测结果
                        )
                    )
            stacked_saved=stacked_saved.t()
            save_to_csv(stacked_saved)

        print("--------------dev完成-----------------")
        

In [18]:
train()

------------Epoch: 0 ----------------
epoth: 0, iter_num: 100, loss: 0.0439, 2.22%
epoth: 0, iter_num: 200, loss: 0.0872, 4.44%
epoth: 0, iter_num: 300, loss: 0.0342, 6.67%
epoth: 0, iter_num: 400, loss: 0.0384, 8.89%
epoth: 0, iter_num: 500, loss: 0.0602, 11.11%
epoth: 0, iter_num: 600, loss: 0.0344, 13.33%
epoth: 0, iter_num: 700, loss: 0.0481, 15.56%
epoth: 0, iter_num: 800, loss: 0.0237, 17.78%
epoth: 0, iter_num: 900, loss: 0.0377, 20.00%
epoth: 0, iter_num: 1000, loss: 0.1092, 22.22%
epoth: 0, iter_num: 1100, loss: 0.0614, 24.44%
epoth: 0, iter_num: 1200, loss: 0.0865, 26.67%
epoth: 0, iter_num: 1300, loss: 0.0143, 28.89%
epoth: 0, iter_num: 1400, loss: 0.0177, 31.11%
epoth: 0, iter_num: 1500, loss: 0.0664, 33.33%
epoth: 0, iter_num: 1600, loss: 0.0104, 35.56%
epoth: 0, iter_num: 1700, loss: 0.0592, 37.78%
epoth: 0, iter_num: 1800, loss: 0.0368, 40.00%
epoth: 0, iter_num: 1900, loss: 0.0221, 42.22%
epoth: 0, iter_num: 2000, loss: 0.0725, 44.44%
epoth: 0, iter_num: 2100, loss: 0.0

In [19]:
from scipy import stats
df_true=pd.read_csv('/kaggle/input/edos-dev-task/edos_labelled.csv',engine='python')

df_pred=pd.read_csv('/kaggle/working/test_a_1.csv',engine='python')
df_pred.info()
def func(df):
    return stats.mode(df.values)[0][0]
    # 给DataFrame应用函数

df_pred.loc[df_pred['label_pred'].notnull(),'label_pred'] = df_pred['label_pred']
df_pred= df_pred.groupby(by='rewire_id').agg(func).reset_index()

df_true.info()
df_data=pd.merge(df_pred,df_true,on='rewire_id')
df_data.info()
df=df_data.sort_values(by=['rewire_id'],ascending=False)
df.info()
print(df.head())
df.loc[df['label_sexist']=='not sexist','label_true_0']=0
df.loc[df['label_sexist']=='sexist','label_true_0']=1
df.loc[df['label_pred']=='not sexist','label_pred_0']=0
df.loc[df['label_pred']=='sexist','label_pred_0']=1

print(df.head())

file1=np.array(df)
pred=[x[8] for x in file1]#pred
true=[x[7] for x in file1]#true
#print(type(pred))

stacked=torch.stack(
                (
                    torch.as_tensor(list(map(int,pred))).to(device),
                    torch.as_tensor(list(map(int,true))).to(device)
                )
            )
stacked=stacked.t()
#print(stacked)
calculate_f1(stacked)
print('---------------f1_score-----------------------')
print(f1_score(true,pred, average='macro'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   rewire_id   2000 non-null   object
 1   label_pred  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   rewire_id       20000 non-null  object
 1   text            20000 non-null  object
 2   label_sexist    20000 non-null  object
 3   label_category  20000 non-null  object
 4   label_vector    20000 non-null  object
 5   split           20000 non-null  object
dtypes: object(6)
memory usage: 937.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----